In [1]:
from pyspark.sql import SparkSession

# since everyone will be using cluster at the same time
# let's make sure that everyone has resource. that is why 
# the configuration uses dynamic resource allocation and
# maximum 1 executor 
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true")\
    .config("spark.dynamicAllocation.maxExecutors", "1")\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/22 08:31:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
songs_df = spark.read.load("./train_triplets.txt",
                     format="csv", sep="\t", inferSchema="true", 
                     header="false")


In [3]:
songs_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)



In [6]:
songs_df.createOrReplaceTempView("songs")
songs_df = songs_df.withColumnRenamed("_c0", "user")\
                   .withColumnRenamed("_c1", "song")\
                   .withColumnRenamed("_c2", "play_count")
songs_df.printSchema()

root
 |-- user: string (nullable = true)
 |-- song: string (nullable = true)
 |-- play_count: integer (nullable = true)



In [7]:
played_more_than_10_times = spark.sql("select song from songs where play_count > 10")

In [8]:
played_more_than_10_times.count()

2043582

In [17]:
username = "b80344d063b5ccb3212f76538f3d9e43d87dca9e"
played_by_user = spark.sql(f"select song from songs where user=\"{username}\"")
played_by_user.count()

104

In [16]:
first_ten_entries = spark.sql(f"select user from songs limit 10")
print(first_ten_entries.collect()[0])

Row(user='b80344d063b5ccb3212f76538f3d9e43d87dca9e')


In [ ]:
username = "b80344d063b5ccb3212f76538f3d9e43d87dca9e"
played_by_user_more_than_10_times = spark.sql(f"select song from songs where user=\"{username}\" and play_count > 10")
played_by_user_more_than_10_times.count()

## Yelp dataset

In [ ]:
business = spark.read.json("./yelp-dataset/yelp_academic_dataset_business.json")
reviews = spark.read.json("./yelp-dataset/yelp_academic_dataset_review.json")
users = spark.read.json("./yelp-dataset/yelp_academic_dataset_user.json")
business.createOrReplaceTempView("business")
reviews.createOrReplaceTempView("reviews")
users.createOrReplaceTempView("users")

In [ ]:
spark.sql("select state, count(state) as count from business group by state order by count(state) desc").show()

In [ ]:
spark.sql("""
select count(distinct(*)) from (
    select explode(split(categories, \",\s*\")) as category from business
)
""").show()

In [ ]:
spark.sql("""
select category, count(category) from 
    (
        select explode(split(categories, \",\s*\")) as category 
        from business where city=\"Phoenix\"
    )
group by category order by count(category) desc limit 10
""").show()

In [ ]:
spark.sql("""
select 
    count(*) as friend_count 
from 
    users 
where 
    size(split(friends, \",\s*\")) > 1000
""").show()

In [ ]:
spark.sql("""
with business_ratings as (
    select 
        business_id, year(to_date(date)) as year, avg(stars) as rating 
    from 
        reviews group by business_id, year(to_date(date))
),
business_2014 as (
    select 
        business_id, rating 
    from 
        business_ratings 
    where 
        year=2014
),
business_2017 as (
    select 
        business_id, rating 
    from 
        business_ratings where year=2017
)
select 
    business_2014.business_id, business_2014.rating, business_2017.rating 
from 
    business_2014 
inner join 
    business_2017 
on 
    business_2014.business_id=business_2017.business_id 
where 
    business_2017.rating < business_2014.rating 
""").show()

In [ ]:
spark.sql("""
with last_review as (
    select user_id, categories, max(to_date(date)) from reviews group by user_id  
)
select explode(split(friends, \",\s*\")) as friend from last_review where category='Chinese restaurant'
""")